<a href="https://colab.research.google.com/github/Saba-G-Haez/NLU-2023-Labs/blob/main/Custom_Dataset_for_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip3 install emoji==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49732 sha256=4df52edbfca788167123c8aadf663e176ed6303184d23b6c0f77a45a73b601c9
  Stored in directory: /root/.cache/pip/wheels/70/2a/7f/1a0012c86b1061c6ee2ed9568b1f830f857a51e8e416452af2
Successfully built emoji


In [2]:
import copy
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

In [3]:
import csv
import urllib.request


# download train instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_text.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]
train_text = [row for row in html]


# download train instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/train_labels.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]    
train_labels = [int(row) for row in html]

In [4]:
# download train instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_text.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]
val_text = [row for row in html]


# download train instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/val_labels.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]    
val_labels = [int(row) for row in html]

In [7]:
# download train instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_text.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]
test_text = [row for row in html]


# download test instances
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/hate/test_labels.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
html = html[:-1]    
test_labels = [int(row) for row in html]

In [15]:
#CHECK SOME BASIC STATS ABOUT THE DATASET
print(len(train_text))
print(len(train_labels))
print(len(val_text))
print(len(val_labels))
print(len(test_text))
print(len(test_labels))

9000
9000
1000
1000
2970
2970


In [17]:
#PRINT SOME ITEMS
for item in train_text[100:115]:
  print(item)
for item in train_labels[100:115]:
  print(item)

@user I bit my toungue when l heard Syrian refugees complain about Rothsey then again , maybe someone can swop them with a high rise 
@user List fails without '50 Things To Think About To Stop You Doing Your Beans' by Kunt and the Gang 
SWEDEN Dentist fined nearly $50,000 for revealing that many adult Muslim migrants are posing as 'unaccompanied refugee children' via @user Swedenstan update... 
The US bill is an illegal endeavour and cannot impact the current situation. It is a part of the “deal of the century” which aims to revoke the Palestinian refugees’ right of return and works hand in hand with the Israeli racist Nation State Law,  
UGH! Fake Christians really out here trying to take away our rights!! IF YOU CARE SO MUCH ABOUT LIFE THEN FIX THE ALREADY BROKEN FOSTER CARE SYSTEM AND WELCOME REFUGEES WITH OPEN ARMS. FOLLOWER OF CHRIST?? WHILE NOT LOVING YOUR NEIGHBORS? YIKES. 
@user Lmao on my grandmother , you never knew me in 2012 so stop while you really ahead hoe 
Could you ple

In [18]:
#CHECK THE LABELS... 
print(sum(train_labels))
print(sum(val_labels))


3783
427


In [19]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel, RobertaTokenizer, RobertaModel

In [29]:
import numpy as np

#CREATE THE TOKENIZER
tokenizer= AutoTokenizer.from_pretrained("vinai/bertweet-base")

#LET'S INVENT SOME NEW FEATURES...
possible_authors = ['John', 'Alice', 'Bob', 'Sarah']
authors = []

for i in range(len(train_text) + len(val_text) + len(test_text)):
  authors.append(possible_authors[np.random.randint(0,4)])

print(authors[100:115])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Alice', 'John', 'John', 'John', 'Alice', 'Sarah', 'Sarah', 'Bob', 'Alice', 'Bob', 'John', 'Alice', 'Bob', 'Bob', 'Alice']


In [32]:
class CustomDataset(Dataset):

    def __init__(self, data, tokenizer, max_len_bert):
      self.data = copy.deepcopy(data)
      self.tokenizer = tokenizer
      self.max_len_bert = max_len_bert
    #WHAT DO WE PUT HERE?

    def __len__(self):
      return len(self.data['labe'])
    #JUST THE LENGTH OF THE DATASET


    def __getitem__(self, idx):
      text = self.data['text'][idx]
      author = self.data['author'][idx]
      label = self.data['lane'][idx]
    #TAKE ONE ITEM FROM THE DATASET
      
      encoding_text = self.tokenoizer.encode_plus(
          text,
          maximum_length=self.max_len_bert,
          add_special_tokens=True,
          padding="max_length",
          return_attention_mask=True,
          return_toke_type_ids=False,
          return_tensor='pt',
          truncation=True
      )

decode = self.tokenizer.convert_ids_to_tokens(encoding_text['input_ids'].flatten()) 

return{
    'text':text,
    'decode':decode,
    'text_input_ids': encoding_text['input_ids'].flatten(),
    'text_attention)mask':encoding_text['attention_mask'].flatten(),
    'author':torch.Tensor([int(self.data['author_list'].index(self.data['author'][idx]))]),
    'labels':torch.Tensor([int(self.data['label'][idx])])
    }

NameError: ignored

In [ ]:
#CREATE THE DATALOADER
def create_data_loader_CustomDataset(data, tokenizer, max_len_bert, batch_size, eval=False):
  ds = CustomDataset(
      data=data,
      tokenizer=tokenizer,
      
  )

In [ ]:
#CREATE ANOTHER DATALOADER...





In [ ]:
#BERT LENGTH, BATCH SIZE AND OTHER THINGS?

#DATALOADERS!

In [ ]:
#lET'S LOOK INSIDE OUR DATALOADER